In [1]:
from __future__ import print_function

import sys
import os
import time
import numpy as np
import mxnet as mx
from mxnet import nd,autograd,init
from mxnet.gluon import Trainer,data as gdata,loss as gloss,nn
from d2l import mxnet as d2l
from IPython import display
import matplotlib.pyplot as plt

from dataset import PartPrimitive
from model import RenderNet
from criterion import BatchIoU
from misc import clip_gradient,gather,scatter_numpy
from programs.label_config import max_param, stop_id
from options import options_train_executor


In [2]:
def adjust_learning_rate(epoch, opt, optimizer):
    """Sets the learning rate to the initial LR decayed by 0.2 every steep step"""
    steps = np.sum(epoch >= np.asarray(opt.lr_decay_epochs))
    if steps > 0:
        new_lr = opt.learning_rate * (opt.lr_decay_rate ** steps)
        optimizer.set_learning_rate(new_lr)


def train(epoch, train_loader, model,loss, optimizer, opt,ctx):
    """
    one epoch training for program executor
    """
    loss_sum,iou_sum,n = 0.0,0.0,0
    for idx, data in enumerate(train_loader):
        start_t = time.time()

        shape, label, param = data
        bsz = shape.shape[0]
        n_step = label.shape[1]
        #print("label.shape:",label)
        #print("n_step:",n_step,"bsz:",bsz,"stop_id:",stop_id)
        
        index = np.array(list(map(lambda x: n_step, label)))-1
        #index = label
        
        # add noise during training, making the executor accept
        # continuous output from program generator
        label = label.reshape(-1,1).asnumpy()
        pgm_vector = 0.1 * np.random.uniform(0,1,(bsz * n_step, stop_id))
        pgm_noise = 0.1 *np.random.uniform(0,1,label.shape)
        pgm_value = 1 - pgm_noise
        #print('pgm_val.shape:',pgm_value.shape,'label.shape:',label.shape,'label.shape:',label.shape)
        pgm_vector = scatter_numpy(pgm_vector,1,label,pgm_value).reshape(bsz,n_step,stop_id)
        
        
        param_noise = nd.random_uniform(0,1,shape=param.shape)
        param_vector = param + 0.6 * (param_noise - 0.5)
        #print("param_vector.shape:",param_vector.shape)
        gt = shape.as_in_context(ctx)
        #print(pgm_vector.dtype)
        index = nd.from_numpy(index).astype('int64').as_in_context(ctx)
        pgm_vector = nd.from_numpy(pgm_vector).astype('float32').as_in_context(ctx)
        param_vector = param_vector.as_in_context(ctx)


        with autograd.record():
            pred = model(pgm_vector, param_vector, index)
            scores = nd.log_softmax(pred,axis=1)
            pred0 = scores[:,0].squeeze()*opt.n_weight
            pred1 = scores[:,1].squeeze()*opt.p_weight
            l = -nd.where(gt, pred1, pred0).mean((1,2,3))
            #l = -(nd.pick(scores1, gt, axis=1, keepdims=True)*opt.n_weight
            #    +nd.pick(scores2,(1-gt), axis=1, keepdims=True)*opt.p_weight).mean((1,2,3,4))
        l.backward()
                                        
        #clip_gradient(optimizer, opt.grad_clip)
        #optimizer._allreduce_grads();

        optimizer.step(l.shape[0],ignore_stale_grad=True)
        
        l = l.mean().asscalar()
        
        pred = nd.softmax(pred,axis = 1)
        pred = pred[:, 1, :, :, :]
        s1 = gt.reshape(-1, 32, 32, 32).astype('float32').as_in_context(mx.cpu())
        s2 = pred.squeeze().as_in_context(mx.cpu())
        #print(s2.shape)
        s2 = (s2 > 0.5)

        batch_iou = BatchIoU(s1, s2)
        iou = batch_iou.mean()
        end_t = time.time()
        loss_sum+=l
        n+=1
        iou_sum+=iou

        if idx % (opt.info_interval * 10) == 0:
            print("Train: epoch {} batch {}/{}, loss13 = {:.3f}, iou = {:.3f}, time = {:.3f}"
                  .format(epoch, idx, len(train_loader), l, iou, end_t - start_t))
            sys.stdout.flush()


def validate(epoch, val_loader, model, loss, opt, ctx, gen_shape=False):

    # load pre-fixed randomization
    try:
        rand1 = np.load(opt.rand1)
        rand2 = np.load(opt.rand2)
        rand3 = np.load(opt.rand3)
    except:
        rand1 = np.random.rand(opt.batch_size * opt.seq_length, stop_id).astype(np.float32)
        rand2 = np.random.rand(opt.batch_size * opt.seq_length, 1).astype(np.float32)
        rand3 = np.random.rand(opt.batch_size, opt.seq_length, max_param - 1).astype(np.float32)
        np.save(opt.rand1, rand1)
        np.save(opt.rand2, rand2)
        np.save(opt.rand3, rand3)

    generated_shapes = None
    original_shapes = None
    
    loss_sum,iou_sum,n = 0.0,0.0,0
    for idx, data in enumerate(val_loader):
        start_t = time.time()

        shape, label, param = data

        bsz = shape.shape[0]
        n_step = label.shape[1]
        index = np.array(list(map(lambda x: n_step, label)))
        index = index - 1

        # add noise during training, making the executor accept
        # continuous output from program generator
        label = label.reshape(-1,1).asnumpy()
        pgm_vector = 0.1*rand1
        pgm_noise = 0.1*rand2
        pgm_value = np.ones(label.shape) - pgm_noise
        #print('pgm_val.shape:',pgm_value.shape,'label.shape:',label.shape,'label.shape:',label.shape)
        pgm_vector = scatter_numpy(pgm_vector,1,label,pgm_value).reshape(bsz,n_step,stop_id)

        param_noise = nd.from_numpy(rand3)
        #print(param.shape,param_noise.shape)
        param_vector = param + 0.6 * (param_noise - 0.5)
        
        
        gt = shape.astype('float32').as_in_context(ctx)
        index = nd.from_numpy(index).astype('int64').as_in_context(ctx)
        pgm_vector = nd.from_numpy(pgm_vector).as_in_context(ctx)
        param_vector = param_vector.as_in_context(ctx)
        #prediction
        pred = model(pgm_vector, param_vector, index)
        scores = nd.log_softmax(pred,axis=1)
        gt_ones = gt.sum()
        gt_zeros = bsz*(32**3)-gt_ones;
        pred0 = scores[:,0].squeeze()*opt.n_weight
        pred1 = scores[:,1].squeeze()*opt.p_weight
        l = -nd.where(gt, pred1, pred0).sum()/(gt_ones*opt.p_weight+gt_zeros*opt.n_weight)
        #print(pred2.dtype,gt.dtype)
        #l = loss(pred,gt,sample_weight = nd.array([opt.n_weight,opt.p_weight]))

        l = l.mean().asscalar()
        pred = nd.softmax(pred,axis=1)
        pred = pred[:, 1, :, :, :]
        s1 = gt.reshape(-1, 32, 32, 32).as_in_context(mx.cpu())
        s2 = pred.squeeze().as_in_context(mx.cpu())
        s2 = (s2 > 0.5)

        batch_iou = BatchIoU(s1, s2)
        iou = batch_iou.mean()
        loss_sum+=l
        n+=1
        iou_sum+=iou
        
        if(idx+1)%5==0 and gen_shape:
            if original_shapes is None:
                original_shapes = s1.expand_dims(axis=0)
                generated_shapes = s2.expand_dims(axis=0)
            else:
                original_shapes = nd.concat(original_shapes,s1.expand_dims(axis=0),dim=0)
                generated_shapes = nd.concat(generated_shapes,s2.expand_dims(axis=0),dim=0)
        end_t = time.time()

        if (idx + 1) % opt.info_interval == 0:
            print("Test: epoch {} batch {}/{}, loss13 = {:.3f}, iou = {:.3f}, time = {:.3f}"
                  .format(epoch, idx + 1, len(val_loader), l, iou, end_t - start_t))
            sys.stdout.flush()
        if(idx+1>len(val_loader)/10):
            
            break;     

    return generated_shapes, original_shapes




In [3]:

opt = options_train_executor.parse()

print('===== arguments: program executor =====')
for key, val in vars(opt).items():
    print("{:20} {}".format(key, val))
print('===== arguments: program executor =====')



===== arguments: program executor =====
learning_rate        0.001
lr_decay_epochs      [20, 25]
lr_decay_rate        0.2
weight_decay         0
beta1                0.5
beta2                0.999
grad_clip            0.1
epochs               30
info_interval        10
save_interval        1
program_size         21
input_encoding_size  128
program_vector_size  128
nc                   2
rnn_size             256
num_layers           1
drop_prob_lm         0
seq_length           3
max_param            7
batch_size           256
num_workers          4
train_file           ./data/train_blocks.h5
val_file             ./data/val_blocks.h5
model_name           program_executor
n_weight             1
p_weight             5
rand1                ./data/rand1.npy
rand2                ./data/rand2.npy
rand3                ./data/rand3.npy
save_folder          ./model/ckpts_program_executor
===== arguments: program executor =====


In [4]:
# build dataloader

print(opt.train_file)
train_set = PartPrimitive(opt.train_file)
train_loader = gdata.DataLoader(
    dataset=train_set,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.num_workers,
)
'''
val_set = PartPrimitive(opt.val_file)
val_loader = gdata.DataLoader(
    dataset=val_set,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.num_workers,
)
'''


# build the model



./data/train_blocks.h5


'\nval_set = PartPrimitive(opt.val_file)\nval_loader = gdata.DataLoader(\n    dataset=val_set,\n    batch_size=opt.batch_size,\n    shuffle=True,\n    num_workers=opt.num_workers,\n)\n'

In [5]:
ctx = d2l.try_gpu()                   
model = RenderNet(opt)
model.initialize(init = init.Xavier(),ctx = ctx)                                        

loss = gloss.SoftmaxCrossEntropyLoss(axis = 1,weight = 5)
#loss = gloss.SigmoidBinaryCrossEntropyLoss(from_sigmoid = True, weight=5)

optimizer = Trainer(model.collect_params(),"adam",
                    {"learning_rate":opt.learning_rate*2,"wd":opt.weight_decay,
                     'beta1':opt.beta1, 'beta2':opt.beta2})
train_from0 = False;
if train_from0:
    if os.path.exists("model/model of executor"):
        model.load_parameters("model/model of executor")
        print("loaded parameter of model_")

In [6]:
a = nd.array([[1,0,1,0,1],[1,0,1,1,0]])
b = nd.arange(10,30).reshape((2,2,5))
print(a,b)
nd.where(a,b[:,1]*5,b[:,0]*1).sum()/34


[[1. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0.]]
<NDArray 2x5 @cpu(0)> 
[[[10. 11. 12. 13. 14.]
  [15. 16. 17. 18. 19.]]

 [[20. 21. 22. 23. 24.]
  [25. 26. 27. 28. 29.]]]
<NDArray 2x2x5 @cpu(0)>



[21.294117]
<NDArray 1 @cpu(0)>

In [7]:
for epoch in range(1, opt.epochs+1):
    adjust_learning_rate(epoch, opt, optimizer)
    print("###################")
    print("training")
    train(epoch, train_loader, model,loss,optimizer, opt,ctx)

    print("###################")
    
    print("testing")
    '''
    gen_shapes, ori_shapes = validate(epoch, val_loader, model,
                                      loss, opt,ctx, val_loss,val_iou, gen_shape=True)
    gen_shapes = (gen_shapes > 0.5)
    gen_shapes = gen_shapes.astype(np.float32)
    iou = BatchIoU(ori_shapes, gen_shapes)
    print("Mean IoU: {:.3f}".format(iou.mean().asscalar()))
    '''
    if epoch % opt.save_interval == 0:
        print('Saving...')
        optimizer.save_states("model/optimizer of executor"),
        model.save_parameters("model/model of executor")

print('Saving...')
optimizer.save_states("model/optimizer of executor"),
model.save_parameters("model/model of executor")

state = {
    'opt': opt,
    'epoch': epoch
}
#save_file = os.path.join(opt.save_folder, 'ckpt_epoch_{epoch}.t7'.format(epoch=epoch))
#nd.save(state, save_file)

###################
training


/home/hzx/anaconda3/envs/shape2prog-master/misc.py:108: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  index[make_slice(index, dim, i)].reshape(1, -1)[0]] for i in range(index.shape[dim])]
/home/hzx/anaconda3/envs/shape2prog-master/misc.py:123: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self[idx] = src[src_idx]


Train: epoch 1 batch 0/3512, loss13 = 0.705, iou = 0.007, time = 0.862


/home/hzx/anaconda3/envs/shape2prog-master/misc.py:108: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  index[make_slice(index, dim, i)].reshape(1, -1)[0]] for i in range(index.shape[dim])]
/home/hzx/anaconda3/envs/shape2prog-master/misc.py:123: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self[idx] = src[src_idx]


Train: epoch 1 batch 100/3512, loss13 = 0.077, iou = 0.001, time = 0.426
Train: epoch 1 batch 200/3512, loss13 = 0.054, iou = 0.012, time = 0.405
Train: epoch 1 batch 300/3512, loss13 = 0.054, iou = 0.009, time = 0.401
Train: epoch 1 batch 400/3512, loss13 = 0.048, iou = 0.004, time = 0.409
Train: epoch 1 batch 500/3512, loss13 = 0.067, iou = 0.008, time = 0.413
Train: epoch 1 batch 600/3512, loss13 = 0.047, iou = 0.001, time = 0.434
Train: epoch 1 batch 700/3512, loss13 = 0.050, iou = 0.013, time = 0.404
Train: epoch 1 batch 800/3512, loss13 = 0.055, iou = 0.005, time = 0.423
Train: epoch 1 batch 900/3512, loss13 = 0.042, iou = 0.005, time = 0.404
Train: epoch 1 batch 1000/3512, loss13 = 0.054, iou = 0.011, time = 0.413
Train: epoch 1 batch 1100/3512, loss13 = 0.040, iou = 0.008, time = 0.404
Train: epoch 1 batch 1200/3512, loss13 = 0.032, iou = 0.019, time = 0.416
Train: epoch 1 batch 1300/3512, loss13 = 0.048, iou = 0.025, time = 0.417
Train: epoch 1 batch 1400/3512, loss13 = 0.047,

...

Train: epoch 16 batch 400/3512, loss13 = 0.007, iou = 0.777, time = 0.426
Train: epoch 16 batch 500/3512, loss13 = 0.005, iou = 0.852, time = 0.400
Train: epoch 16 batch 600/3512, loss13 = 0.004, iou = 0.803, time = 0.410
Train: epoch 16 batch 700/3512, loss13 = 0.005, iou = 0.828, time = 0.420
Train: epoch 16 batch 800/3512, loss13 = 0.003, iou = 0.840, time = 0.420
Train: epoch 16 batch 900/3512, loss13 = 0.004, iou = 0.849, time = 0.419
Train: epoch 16 batch 1000/3512, loss13 = 0.009, iou = 0.704, time = 0.434
Train: epoch 16 batch 1100/3512, loss13 = 0.004, iou = 0.843, time = 0.419
Train: epoch 16 batch 1200/3512, loss13 = 0.004, iou = 0.805, time = 0.401
Train: epoch 16 batch 1300/3512, loss13 = 0.005, iou = 0.830, time = 0.405
Train: epoch 16 batch 1400/3512, loss13 = 0.005, iou = 0.857, time = 0.402
Train: epoch 16 batch 1500/3512, loss13 = 0.004, iou = 0.855, time = 0.408
Train: epoch 16 batch 1600/3512, loss13 = 0.005, iou = 0.786, time = 0.402
Train: epoch 16 batch 1700/3512

Train: epoch 19 batch 400/3512, loss13 = 0.005, iou = 0.750, time = 0.412
Train: epoch 19 batch 500/3512, loss13 = 0.004, iou = 0.862, time = 0.403
Train: epoch 19 batch 600/3512, loss13 = 0.005, iou = 0.776, time = 0.476
Train: epoch 19 batch 700/3512, loss13 = 0.005, iou = 0.848, time = 0.402
Train: epoch 19 batch 800/3512, loss13 = 0.004, iou = 0.845, time = 0.438
Train: epoch 19 batch 900/3512, loss13 = 0.006, iou = 0.831, time = 0.419
Train: epoch 19 batch 1000/3512, loss13 = 0.004, iou = 0.824, time = 0.408
Train: epoch 19 batch 1100/3512, loss13 = 0.004, iou = 0.840, time = 0.419
Train: epoch 19 batch 1200/3512, loss13 = 0.005, iou = 0.823, time = 0.403
Train: epoch 19 batch 1300/3512, loss13 = 0.004, iou = 0.842, time = 0.420
Train: epoch 19 batch 1400/3512, loss13 = 0.003, iou = 0.837, time = 0.404
Train: epoch 19 batch 1500/3512, loss13 = 0.004, iou = 0.814, time = 0.415
Train: epoch 19 batch 1600/3512, loss13 = 0.004, iou = 0.815, time = 0.400
Train: epoch 19 batch 1700/3512

Train: epoch 22 batch 400/3512, loss13 = 0.002, iou = 0.906, time = 0.420
Train: epoch 22 batch 500/3512, loss13 = 0.002, iou = 0.890, time = 0.419
Train: epoch 22 batch 600/3512, loss13 = 0.002, iou = 0.919, time = 0.407
Train: epoch 22 batch 700/3512, loss13 = 0.002, iou = 0.903, time = 0.407
Train: epoch 22 batch 800/3512, loss13 = 0.001, iou = 0.902, time = 0.408
Train: epoch 22 batch 900/3512, loss13 = 0.002, iou = 0.905, time = 0.410
Train: epoch 22 batch 1000/3512, loss13 = 0.002, iou = 0.912, time = 0.407
Train: epoch 22 batch 1100/3512, loss13 = 0.002, iou = 0.907, time = 0.409
Train: epoch 22 batch 1200/3512, loss13 = 0.002, iou = 0.914, time = 0.408
Train: epoch 22 batch 1300/3512, loss13 = 0.002, iou = 0.909, time = 0.404
Train: epoch 22 batch 1400/3512, loss13 = 0.002, iou = 0.898, time = 0.402
Train: epoch 22 batch 1500/3512, loss13 = 0.002, iou = 0.909, time = 0.424
Train: epoch 22 batch 1600/3512, loss13 = 0.002, iou = 0.898, time = 0.404
Train: epoch 22 batch 1700/3512

Train: epoch 25 batch 400/3512, loss13 = 0.001, iou = 0.911, time = 0.410
Train: epoch 25 batch 500/3512, loss13 = 0.002, iou = 0.920, time = 0.410
Train: epoch 25 batch 600/3512, loss13 = 0.002, iou = 0.908, time = 0.401
Train: epoch 25 batch 700/3512, loss13 = 0.002, iou = 0.916, time = 0.398
Train: epoch 25 batch 800/3512, loss13 = 0.002, iou = 0.910, time = 0.412
Train: epoch 25 batch 900/3512, loss13 = 0.002, iou = 0.904, time = 0.418
Train: epoch 25 batch 1000/3512, loss13 = 0.002, iou = 0.932, time = 0.401
Train: epoch 25 batch 1100/3512, loss13 = 0.002, iou = 0.886, time = 0.416
Train: epoch 25 batch 1200/3512, loss13 = 0.002, iou = 0.909, time = 0.425
Train: epoch 25 batch 1300/3512, loss13 = 0.002, iou = 0.899, time = 0.406
Train: epoch 25 batch 1400/3512, loss13 = 0.002, iou = 0.917, time = 0.401
Train: epoch 25 batch 1500/3512, loss13 = 0.002, iou = 0.899, time = 0.402
Train: epoch 25 batch 1600/3512, loss13 = 0.002, iou = 0.920, time = 0.405
Train: epoch 25 batch 1700/3512

Train: epoch 28 batch 400/3512, loss13 = 0.002, iou = 0.926, time = 0.408
Train: epoch 28 batch 500/3512, loss13 = 0.002, iou = 0.885, time = 0.425
Train: epoch 28 batch 600/3512, loss13 = 0.001, iou = 0.924, time = 0.405
Train: epoch 28 batch 700/3512, loss13 = 0.002, iou = 0.906, time = 0.399
Train: epoch 28 batch 800/3512, loss13 = 0.002, iou = 0.925, time = 0.412
Train: epoch 28 batch 900/3512, loss13 = 0.002, iou = 0.904, time = 0.402
Train: epoch 28 batch 1000/3512, loss13 = 0.002, iou = 0.914, time = 0.406
Train: epoch 28 batch 1100/3512, loss13 = 0.002, iou = 0.922, time = 0.439
Train: epoch 28 batch 1200/3512, loss13 = 0.002, iou = 0.917, time = 0.418
Train: epoch 28 batch 1300/3512, loss13 = 0.002, iou = 0.914, time = 0.418
Train: epoch 28 batch 1400/3512, loss13 = 0.002, iou = 0.906, time = 0.417
Train: epoch 28 batch 1500/3512, loss13 = 0.001, iou = 0.922, time = 0.411
Train: epoch 28 batch 1600/3512, loss13 = 0.002, iou = 0.926, time = 0.424
Train: epoch 28 batch 1700/3512

optimizer.save_states("model/model of executor"),
model.save_parameters("model/model of executor")

In [8]:
val_set = PartPrimitive(opt.val_file)
val_loader = gdata.DataLoader(
    dataset=val_set,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.num_workers,
)
print("testing")
for epoch in range(1, opt.epochs+1):
    gen_shapes, ori_shapes = validate(epoch, val_loader, model,
                                      loss, opt,ctx, val_loss,val_iou, gen_shape=True)
    gen_shapes = (gen_shapes > 0.5)
    gen_shapes = gen_shapes.astype(np.float32)
    iou = BatchIoU(ori_shapes, gen_shapes)
    print("Mean IoU: {:.3f}".format(iou.mean()))


testing


NameError: name 'val_loss' is not defined

In [ ]:
gen_shapes, ori_shapes = validate(epoch, val_loader, model,
                                      loss, opt,ctx, val_loss,val_iou, gen_shape=True)
gen_shapes = (gen_shapes > 0.5)
gen_shapes = gen_shapes.astype(np.float32)
iou = BatchIoU(ori_shapes, gen_shapes)
print("Mean IoU: {:.3f}".format(iou.mean().asscalar()))